<a href="https://colab.research.google.com/github/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/Chapter11_Using_Convolutional_and_Recurrent_Methods_for_Sequence_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11: Using Convolutional and Recurrent Methods for Sequence Models


![overview](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/overview.png?raw=true)

Hi, chào mọi người, tụi mình lại gặp nhau rồi. 😊  

Nếu như ở chương 9 tụi mình đã tìm hiểu về dữ liệu chuỗi thời gian là gì cũng như các đặc tính và phương pháp thống kê, dự đoán đơn giản, thì sang đến chương 10, chúng ta đã biết thêm được cách xử lý dữ liệu đưa vào mô hình để dự đoán. Tuy nhiên, các mô hình này vẫn còn rất đơn giản khi chỉ với vài lớp tuyến tính.  

Vậy nên, đến với chương 11 này, tụi mình sẽ cùng khám phá cách ứng dụng các lớp tích chập cũng như hồi quy, LSTM vào bên trong mô hình dự đoán nha. Chúng sẽ làm tăng độ hiệu quả của kết quả dự đoán lên đáng kinh ngạc á. 🚀  

> **Hả, lớp tích chập á?**  

Đúng rồi, bạn không đọc nhầm đâu, chính xác là các lớp tích chập á. Chúng không chỉ áp dụng với dữ liệu dạng ảnh 2D hay tensor nhiều chiều, mà chúng còn ứng dụng cả cho dữ liệu dạng chuỗi 1D (univariate time series) nữa á. 🌟  

---  

*Lưu ý: Dữ liệu chuỗi thời gian cũng có thể ở dạng tensor nhiều chiều với mỗi chiều được tính là một đặc trưng, nó là **multivariate time series** mà chúng ta từng học á.* 📊

# Convolution for Sequence Data  

Tụi mình nhớ lại bài học cũ về lớp tích chập nhé.  

> **Có phải điểm mấu chốt có ích mà lớp tích chập mang lại là các giá trị thông qua bộ lọc không nè.**  

Khi đầu vào là một ma trận hay tensor đại diện cho một tấm ảnh thì sẽ có các bộ lọc hình vuông quét qua bức ảnh, nhờ đó mà ta có thể thu được các đặc trưng quan trọng của hình ảnh, giúp nắm bắt được các đặc điểm cục bộ của bức hình.  

Vậy bây giờ chúng ta cũng có thể tiến hành ứng dụng tương tự ra với dữ liệu dạng chuỗi nha. Nguyên lý vẫn là dựa trên việc bộ lọc quét qua và tiến hành trích xuất các đặc trưng quan trọng. Tuy nhiên giờ đây **kích thước của từng bộ lọc chỉ đơn giản là mảng một chiều** chứ không còn 2 chiều như trước nữa nha.  

Dưới đây là hình ảnh ví dụ minh họa nha. 🌟📊

![conv1d](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/conv1d.png?raw=true)

Như mọi người thấy á, tụi mình có một chuỗi dữ liệu thời gian là:  

`[4, 8, 15, 16, 23, 42, 51, 64, 99, -1]`  

Thì bộ lọc của tụi mình là mảng 1 chiều: `[-0.5, 1, -0.5]`  

Sau khi bộ lọc quét qua thì tiến hành tổng hợp giá trị tại 3 bước thời gian đầu tiên lại thành **-1.5** á. 💡  

Sau đó bộ lọc lại tiếp tục trượt, tiến lên một bước và tiến hành trích xuất tiếp đặc trưng như trên. 🔄

![conv1d2](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/conv1d2.png?raw=true)

Vậy là với phương pháp này, tụi mình đã có thể nắm bắt được các mẫu, đặc trưng quan trọng giữa các giá trị với nhau. Bây giờ thì mọi người đã thấy nó cũng tương tự, chả khác gì mấy với việc áp dụng lên dữ liệu hình ảnh ha. 🌟  

Thông qua việc sử dụng các lớp tích chập xử lý dữ liệu chuỗi thời gian thì quá trình suy luận và học tập của mô hình cũng được cải thiện rõ rệt. Khả năng nắm bắt được tính **autocorrelation** của dữ liệu cũng được tăng lên rất nhiều. Qua đó kết quả đầu ra cũng có phần cải thiện hơn nha. 📈  

Bây giờ tụi mình sẽ tiến hành tạo lại dữ liệu và thiết kế mô hình với các lớp tích chập nha.  

Mình sẽ tạo lại dữ liệu như chương trước nha. 🔄

In [ ]:
# Thêm các thư viện
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Dropout, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber
import time as timer
from datetime import timedelta
import pandas as pd
import math
import regex as re
from sklearn.metrics import mean_absolute_error

In [ ]:
# Tạo một MirroredStrategy để sử dụng tất cả các GPU có sẵn
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
#@title Tạo lại dữ liệu chuỗi thời gian (mọi người ấn vào để coi chi tiết code bên trong nha)
def plot_series(time, series, format="-", start=0, end=None):
  """
  Hàm này dùng để vẽ dữ liệu chuỗi thời gian trong một khung thời gian

  Tham số đầu vào:
  - time: khoảng thời gian xảy ra
  - series: chuỗi giá trị tương ứng trong thời gian đó
  - start: thời gian bắt đầu
  - end: thời gian kết thúc
  """

  plt.plot(time[start:end], series[start:end], format)
  plt.xlabel("Time")
  plt.ylabel("Value")
  plt.grid(True)


def trend(time, slope=0):
  """
  Hàm này giúp điều hướng xu hướng của dữ liệu, giống với phương trình bậc 1

  Tham số đầu vào:
  - time: khoảng thời gian xảy ra
  - slope: độ dốc của dữ liệu.
    + Nếu giá trị dương thì đường thẳng hưởng lên, ta có xu hướng Uptrend
    + Nếu giá trị âm thì chúng ta có xu hướng Downtrend

  Kết quả trả về:
  - series: danh sách giá trị theo xu hướng thời gian
  """
  return slope * time


def seasonal_pattern(season_time):
  """
  Hàm này dùng để tạo ra mẫu dữ liệu trong 1 mùa.
  Ở đây mình chia làm 2 pha tính, logic tính toán như sau nha:
  - Nếu thời gian mùa < 0.4, tiến hành tính theo dao động cosine: np.cos(season_time *2 *np.pi)
  - Nếu thời gian mùa > 0.4, tiến hành tính theo hàm suy giảm mũ:  1 / np.exp(3 * season_time)

  Tham số đầu vào:
  - thời gian của mùa

  Kết quả trả về:
  - res: kết quả tính ra được từ logic tính toán nằm trong khoảng [-1, 1]
  """
  res = np.where(season_time<0.4,
                 np.cos(season_time * 2 * np.pi),
                 1 / np.exp(3 * season_time))
  return res

def seasonality(time, period, amplitude=1, phase=0):
  """
  Hàm này để lặp lại, tạo nên thời gian cho các mùa nha.
  Logic tính toán sẽ như sau:
  - time % period sẽ ra giá trị trong khoảng [0, period - 1] -> tạo ra được các lần lặp thời gian của mùa [0->period-1, 0->period-1,...]
  - / period sẽ tiến hành chuẩn hóa lại thời gian của mùa về khoảng [0, 1]
  - Chạy seasion time vô hàm seasonal_pattern để tạo ra chuỗi giá trị tương ứng với dãy thời gian của các mùa, giá trị nằm trong khoảng [-1, 1]
  - amplitude * seasonal_pattern, ta tiến hành nhân biên độ dao động với chuỗi giá trị thì giá trị cuối cùng sẽ trong khoảng [-amplitude, amplitude]

  Tham số đầu vào:
  - time: khoảng thời gian xảy ra
  - period: khoảng thời gian của một mùa
  - amplitude: biên độ dao động tối đa của dữ liệu, ví dụ mình để là 15 thì sẽ dao động trong khoảng giá trị từ -15 đến 15
  - phase: độ lệch pha, ví dụ nếu ta gán 15, thay vì mùa vụ bắt đầu từ thời điểm t = 0 thì bị lệch đi 15 bước thời gian tức bắt đầu vào t = 15

  Kết quả trả về:
  - res: dữ liệu mang tính chu kỳ, mùa vụ
  """
  season_time = ((time + phase) % period) / period
  res = amplitude * seasonal_pattern(season_time)
  return res


def noise(time, noise_level=1, seed=None):
  """
  Hàm này để tạo tính nhiễu cho dữ liệu
  Logic tính toán như sau:
  - rnd.rand(len(time)) trả về một dãy giá trị có độ lớn trong khoảng [0,1),
   số lượng giá trị bằng số bước thời gian
  - Sau đó nhân với noise_level sẽ ra giá trị trong khoảng [0, noise_level)

  Tham số đầu vào:
  - time: khoảng thời gian xảy ra.
  - noise_level: cấp độ nhiễu của dữ liệu
  - seed: nếu để giá trị khác None, thì ở các lần chạy sau,
  khi giá trị seed bằng nhau thì giá trị ngẫu nhiên tạo ra cũng bằng nhau

  Kết quả đầu ra:
  - res: dữ liệu mang tính nhiếu
  """
  rnd = np.random.RandomState(seed)
  res = rnd.randn(len(time)) * noise_level
  return res

# Bắt đầu tạo dữ liệu
time = np.arange(4 * 365 + 1, dtype="float32")
# series = trend(time, 0.1)
# baseline = 10
# amplitude = 15
# slop = 0.09
# noise_level = 15
# period = 360

series = trend(time, 0.1)
baseline = 10
amplitude = 20
slop = 0.09
noise_level = 5
period = 360

series = baseline + trend(time, slop)
series += seasonality(time, period=period, amplitude=amplitude)
series += noise(time, noise_level, seed=42)
plot_series(time, series)

# Chia dữ liệu train, val ra
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

print("Tiến hành tạo dữ liệu thành công!")
print(f"Kích thước tập train: {len(x_train)}")
print(f"Kích thước tập val: {len(x_valid)}")

In [ ]:
# Tiến hành tạo "windowed dataset"
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis=-1) # Tiến hành thêm một chiều mới để chuyển thành tensor có kích thước là (n, 1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [ ]:
# Tiến hành tạo dataset để đưa vào mô hình
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

train_dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
val_dataset = windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)

In [ ]:
with strategy.scope():
  model = Sequential([
      Conv1D(filters=128, kernel_size=3, # Có 128 bộ lọc với kích thước là 3
            strides=1, padding='causal', # Bước nhảy là 1,
            activation='relu',
            input_shape=[None, 1]), # Vì dữ liệu của mình là univariate nên số lớp là 1 nha.
      Dense(28, activation='relu'),
      Dense(10, activation='relu'),
      Dense(1),
  ])

  optimizer = SGD(learning_rate=1e-5, momentum=0.5)
  model.compile(loss='mse', optimizer=optimizer)
model.summary()

Ở đây mình giải thích một tí về tham số mới `padding = 'causal'` nha. Cái này thường được sử dụng nhiều với dữ liệu thời gian á. ⏳  

Vì cơ chế hoạt động tương tự với lớp tích chập xử lý ảnh, nên phương pháp này sẽ không tránh khỏi việc kích thước sau khi qua bộ lọc bị giảm đi do thất thoát giá trị tại các đường biên ha.  

Vậy chúng ta cần phương pháp gì đó để xử lý được việc này, và đó chính là **'causal'**. Cơ chế của chúng sẽ hoạt động như sau:  
> Giá trị dự đoán, đầu ra tại thời điểm **t** sẽ chỉ phụ thuộc vào các giá trị tại thời điểm **t** và trước đó **t-1, t-2, t-3,...**.  

Qua đó, ta có thể đảm bảo được việc sẽ không sử dụng thông tin tương lai để đưa vào dự đoán. ✅  

**Kích thước padding** hay số lượng giá trị thêm vào sẽ bằng `kernel_size - 1`.  

### Ví dụ:  
- **Dữ liệu đầu vào**: `[1, 2, 3, 4, 5]`  
- **Kích thước bộ lọc**: `3`  

Lúc này, tụi mình sẽ có số lượng padding vào chuỗi là: `3 - 1 = 2`.  

Vậy **chuỗi đầu vào sau khi đệm** là: `[0, 0, 1, 2, 3, 4, 5]`.  

Sau khi đã đệm xong, thì bộ lọc sẽ tiến hành trượt qua và trích xuất đặc trưng nha. 🎯

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian chạy: {timedelta(seconds=end_time - start_time)}")

In [ ]:
# Vẽ biểu đồ huấn luyện
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Đây là kết quả của mình nha.

![conv1d](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/conv1d_model.png?raw=true)

Uầy, nhìn kết quả có vẻ khá khả quan ha mọi người. Ở đây mình định nghĩa nhanh luôn một hàm để dự đoán nha.

In [ ]:
def model_forecast(model, series, window_size):
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size))
  ds = ds.batch(32).prefetch(1)
  forecast = model.predict(ds)
  return forecast

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
result = forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plot_series(time_valid, x_valid)
plot_series(time_valid, result)

Đây là kết quả dự đoán của mình nha.

![prediction_conv1d](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_conv1d_model.png?raw=true)

Nhìn chung kết quả dự đoán của chúng ta vẫn có thể nắm bắt được quy đạo của đường cong, mọi người có thể thấy chúng na ná như nhau. Chỉ là sự chênh lệch về giá trị xảy ra quá lớn.

In [ ]:
# Đánh giá MAE
mae = mean_absolute_error(x_valid, result)
print(f"Chỉ số đánh giá MAE: {mae}")

Kết quả ở lần chạy của mình:  
> **Chỉ số đánh giá MAE**: 4.7682013511657715 🌟  

Tuy nhiên, kết quả đánh giá MAE lại có phần tệ hơn so với trước đó. Nhưng thôi không sao, vấn đề này nằm ở việc chúng ta cần thử nghiệm nhiều lần để tối ưu hóa các siêu tham số của mô hình. 🔄  

Nhìn chung, phương pháp này cũng hiệu quả ha mọi người. 💡

# Experimenting with the Conv1D Hyperparameters

Bây giờ tụi mình sẽ thử nghiệm với các tham số khác nhau của lớp Conv1D để tối ưu hóa mô hình nha, tương tự với chương trước á.

In [ ]:
!pip install keras-tuner

In [ ]:
def build_model(hp):
  model = Sequential([
      Conv1D(filters=hp.Int('units', min_value=128, max_value=256, step=64),
             kernel_size=hp.Int('kernel_size', min_value=3, max_value=9, step=3),
             strides=hp.Int('strides', min_value=1, max_value=3, step=1),
             padding='causal', activation='relu', input_shape=[None,1]),
      Dense(28, activation='relu'),
      Dense(10, activation='relu'),
      Dense(1)
  ])
  optimizer = SGD(learning_rate=1e-5, momentum=0.5)
  model.compile(loss='mse', optimizer=optimizer)
  return model

In [ ]:
from keras_tuner.tuners import RandomSearch
tuner = RandomSearch(build_model,
                     objective='loss',
                     max_trials=500,
                     executions_per_trial=3,
                     directory='model',
                     project_name='conv1d_search_hyperparameter')

tuner.search_space_summary()

In [ ]:
start_time = timer.time()
tuner.search(train_dataset, epochs=100, validation_data=val_dataset, verbose=0)
end_time = timer.time()

In [ ]:
tuner.results_summary()

Ở đây mình tìm được bộ siêu tham số tốt nhất cho mô hình như sau nha:


```
Trial 006 summary
Hyperparameters:
units: 192
kernel_size: 6
strides: 1
Score: 58.129748026529946
```



Bây giờ tụi mình tiến hành lấy mô hình tốt nhất ra nha.

In [ ]:
model_lowest_loss = tuner.get_best_models(num_models=1)[0]
model_lowest_loss.summary()

In [ ]:
forecast = model_forecast(model_lowest_loss, series[..., np.newaxis], window_size)
result = forecast[split_time - window_size:-1, -1, 0]

In [ ]:
# Biểu diễn biểu đồ:
plot_series(time_valid, x_valid)
plot_series(time_valid, result)

Biểu đồ ở lần chạy của mình nha.

![best_prediction](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/best_predict_conv1d_model.png?raw=true)


In [ ]:
print(f"Đánh giá kết quả theo MAE: {mean_absolute_error(x_valid, result)}")

Kết quả ở lần chạy của mình:
>Đánh giá kết quả theo MAE: 4.881666660308838

Uầy có vẻ kết quả thấp hơn tí trên tập val nhưng mà đây là mô hình tốt nhất dựa trên loss trung bình trên tập train nha.

# Using NASA Weather Data

Bây giờ tụi mình sẽ thử ứng dụng vào bộ dữ liệu thời tiết của NASA nha.

Ở mình sẽ sử dụng bộ **Settle Tacoma (SeaTac)**. Ở bộ dữ liệu này, chúng ta sẽ thử áp dụng các lớp hồi quy RNN cơ bản nha.

![seat_tacoma](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/Seatle_tacoma.png?raw=true)

Mọi người vào đường [link](https://data.giss.nasa.gov/cgi-bin/gistemp/stdata_show_v4.cgi?id=USW00024233&ds=14&dt=1) này rồi kéo xuống dưới cùng sẽ có chỗ tải file CSV nha hoặc cũng có thể chạy đoạn code dưới đây để tải.

*Lưu ý: Nếu đoạn code báo lỗi 404 not found thì là do trang web đang ẩn đi đường link tải, chúng chỉ được tạo ra trong một khoảng thời gian nhất định. Nếu mọi người gặp lỗi thì vô trang web chính ở trên để tải file sau đó quay lại chạy code dưới là được nha.*

In [ ]:
!wget -O data.csv 'https://data.giss.nasa.gov/tmp/gistemp/STATIONS_v4/tmp_USW00024233_14_0_1/station.csv'

Dữ liệu của tụi mình sẽ có dạng như sau:

![df](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/df.png?raw=true)

Ở đây mình sử dụng đoạn code khác với tác giả để trích xuất dữ liệu nha. Mọi người cũng có thể tham khảo code của tác giả ở đây.


```python
 def get_data():
    data_file = "data.csv"
    f = open(data_file)
    data = f.read()
    f.close()
    lines = data.split('\n')
    header = lines[0].split(',')
    lines = lines[1:]
    temperatures=[]
    for line in lines:
        if line:
            linedata = line.split(',')
            linedata = linedata[1:13]
            for item in linedata:
                if item:
                    temperatures.append(float(item))
    series = np.asarray(temperatures)
    time = np.arange(len(temperatures), dtype="float32")
    return time, series
```



In [ ]:
# Code của mình
df = pd.read_csv('data.csv')
df

In [ ]:
# Ở đây mình loại bỏ đi cột year đầu tiên
need_data = df.iloc[0:, 1:]
need_data

In [ ]:
temperatures = []
# Tiến hành duyệt qua các hàng và lấy dữ liệu
for index, row in need_data.iterrows():
  temperatures.extend(row.values)

In [ ]:
# Chuyển định dạng sang numpy
series = np.asarray(temperatures)
time = np.arange(len(temperatures), dtype="float32")

# Chuẩn hóa dữ liệu
series = (series - np.mean(series)) / np.std(series)

In [ ]:
# Tính split_time phù hợp, ở đây mình chọn là 80% nha
split_time = math.ceil(len(time)*0.8)
print(f"Điểm chia dữ liệu phù hợp: {split_time}")
x_train = series[:split_time]
x_val = series[split_time:]
time_train = time[:split_time]
time_valid = time[split_time:]

# Định nghĩa các siêu tham số
window_size = 24
batch_size = 12
shuffle_buffer_size = 48

# Tạo dataset mô hình
train_dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
val_dataset = windowed_dataset(x_val, window_size, batch_size, shuffle_buffer_size)

## Using RNN for Sequence Modeling  

Tiến hành áp dụng **RNN** cho quá trình mô hình hóa chuỗi. Mọi người có thể nhớ lại ở chương 7 về NLP, tụi mình có tìm hiểu về **RNN** rồi nha. Tuy nhiên, lúc đó mình sử dụng dạng cải tiến của nó là **LSTM**, còn bây giờ tụi mình sẽ thử sử dụng **RNN** nha. 🌟

In [ ]:
train_dataset

In [ ]:
with strategy.scope():
  model = Sequential([
      SimpleRNN(100, return_sequences=True,
                input_shape=[None, 1]),
      SimpleRNN(100),
      Dense(1)
  ])

  optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
  model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

Ở đây tụi mình tiếp xúc với một hàm loss mới là **Huber** nha. Nó là sự kết hợp của cả MSE và MAE

Công thức tính loss như sau:

![huber](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/huber.png?raw=true)

Trong đó:  
- **α**: Đại diện cho chỉ số chênh lệch hay hiệu giữa giá trị dự đoán và giá trị thực tế.  
- **δ**: Một ngưỡng cố định để xác định phân chia tính toán dựa trên 2 trường hợp.  

**Huber** có thể khắc phục được những hạn chế cũng như tận dụng ưu điểm của **MSE** và **MAE**:  
- **MSE**: Bị nhạy cảm với các giá trị ngoại lai, biến thiên dẫn đến ảnh hưởng tác động lớn đến kết quả.  
- **MAE**: Ít bị ảnh hưởng bởi ngoại lai nhưng lại có thể làm mất tính ổn định của gradient khi tối ưu hóa vì gradient của MAE là không liên tục, khi đạo hàm sẽ gặp phải điểm gãy tại **x = 0**.  

Do đó, tụi mình sẽ sử dụng **Huber** để cải tiến nha, tiến hành train với **1000 epochs**. 🚀

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện: {str(timedelta(seconds=end_time - start_time))}")

In [ ]:
# Vẽ biểu đồ huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Train loss')
plt.plot(val_loss, label='Val loss')
plt.legend()
plt.show()

Kết quả ở lần chạy của mình nha.

![simple_rnn](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/simple_rnn_model.png?raw=true)

Mọi người lưu ý đoạn code dưới đây giúp mình nha.

Ở đây kích thước đầu ra của mô hình chúng ta sẽ khác với trước, nó chỉ có số chiều là 2 thôi dẫn đến phần lấy result phải được tinh chỉnh lại.


```python
result = forecast[split_time - window_size:-1, 0]
```

Nguyên nhân là vì lớp RNN thứ 2 trong mô hình có `return_sequence=False` khiến cho đầu ra của lớp này chỉ trả về trạng thái cuối chứ không phải một chuỗi, do đó số chiều bị giảm còn 2 so với mô hình trước là 3.



In [ ]:
# Tiến hành dự đoán giá trị
forecast = model_forecast(model, series[...,np.newaxis], window_size)
result = forecast[split_time - window_size:-1, 0]
# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([1200, len(time), -2.5, 2.5]) # Vì dữ liệu khá nhiều nên mình zoom một phần ra cho dễ coi nha.

Biểu đồ dự đoán ở lần chạy của mình.

![prediction_simple_rnn](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_simple_rnn_model.png?raw=true)

In [ ]:
# Đo chỉ số MAE
print(f"Kết quả đánh giá MAE: {mean_absolute_error(x_val, result)}")

Kết quả ở lần chạy của mình:  
> **Kết quả đánh giá MAE**: 0.3146536542989436 🌟  

Nhìn kết quả cũng khá tốt ha mọi người. Chúng vẫn có thể theo được hình dạng, đường cong của giá trị thực tế, chỉ có một chút sai lệch nhỏ ở các đỉnh thôi. Các giá trị dự đoán ở các đỉnh không được sát lắm.  

Bây giờ, tụi mình tiến hành huấn luyện với **1500 epochs** xem thử kết quả ra sao nha, đồng thời kiểm tra giới hạn của mô hình xem đến khoảng nào sẽ gặp phải tình trạng **overfitting**. 📊

In [ ]:
with strategy.scope():
  model = Sequential([
      SimpleRNN(100, return_sequences=True,
                input_shape=[None, 1]),
      SimpleRNN(100),
      Dense(1)
  ])

  optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
  model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

start_time = timer.time()
history = model.fit(train_dataset, epochs=1500, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện: {timedelta(seconds=end_time - start_time)}")

In [ ]:
# Vẽ biểu đồ huấn luyện
# Vẽ biểu đồ huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Train loss')
plt.plot(val_loss, label='Val loss')
plt.legend()
plt.show()

Kết quả ở lần chạy của mình:

![simple_rnn_1500](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/simple_rnn_model_1500.png?raw=true)

Mình nghĩ ở phần này chắc do dữ liệu khác (đã cập nhật thêm dữ liệu hiện tại) dẫn đến kết quả biểu đồ cũng có phần khác.  

Ở trong lần chạy của tác giả thì đến khoảng **epoch 500** đã có tình trạng **overfitting**. Tuy nhiên, khi mình chạy thì có vẻ mô hình vẫn có xu hướng tiếp tục giảm tiếp. Tuy là tốc độ đã bắt đầu chậm đi rồi, nhưng quan sát log thì vẫn đang giảm. 📉  

Chúng ta có thể tiếp tục theo dõi và kiểm tra thêm để đảm bảo kết quả tốt nhất nha. 🚀

Biểu đồ của tác giả:

![root_simple_rnn](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/root_simple_rnn_model.png?raw=true)

In [ ]:
# Tiến hành dự đoán giá trị
forecast = model_forecast(model, series, window_size)
result = forecast[split_time-window_size:-1, 0]

# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([1200, len(time), -2.5, 2.5])

Biểu đồ ở lần chạy của mình:


![best_prediction_simple_rnn](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/best_prediction_simple_rnn_model.png?raw=true)

In [ ]:
# Đo chỉ số MAE
print(f"Kết quả đánh giá MAE: {mean_absolute_error(x_val, result)}")

Uầy, kết quả bất ngờ thật đấy:  
- **100 epochs**: Kết quả đánh giá MAE: **0.3146536542989436**  
- **1500 epochs**: Kết quả đánh giá MAE: **0.20764662626308328** 🎉  

Ở các đỉnh, giá trị dự đoán cũng có phần sát hơn rất là nhiều. Mô hình đã thật sự cải thiện rõ rệt và đang tiếp tục, chưa có dấu hiệu **overfitting**. 🌟  

Tiếp tục thử nghiệm thêm để xem giới hạn tối ưu của mô hình đến đâu nha! 🚀

# Exploring a Larger Dataset

Bây giờ tụi mình sẽ thử mở rộng bộ dữ liệu ra hơn nha. Ở đây mình sẽ truy cập vào [KNMI Climate Explorer](https://climexp.knmi.nl/start.cgi) để lấy [dữ liệu thời tiết](https://hadleyserver.metoffice.gov.uk/hadcet/data/meantemp_daily_totals.txt) khu vực trung tâm nước Anh trong giai đoạn từ năm 1772 đến 2024 nha.

In [ ]:
!wget -O daily_temp.txt 'https://hadleyserver.metoffice.gov.uk/hadcet/data/meantemp_daily_totals.txt'

In [ ]:
# Lấy danh sách dữ liệu
def get_data():
  data_path = "daily_temp.txt"
  f = open(data_path, 'r')
  data = f.read()
  f.close()
  lines=data.split('\n')
  temperatures = []
  lines = lines[2:] # Loại bỏ 2 hàng đầu tiên vì chúng chứa tên cột
  for line in lines:
    if line:
      line_data = re.split(r'\s+', line)
      temperatures.append(float(line_data[1]))

  series = np.asarray(temperatures)
  time = np.arange(len(temperatures), dtype='float32')
  return time, series

In [ ]:
time, series = get_data()
print(f"Kích thước dữ liệu: {len(time)}")

Tụi mình cũng tiến hành các bước chia dữ liệu rồi chuẩn bị tương tự như trước nha.

In [ ]:
# Chuẩn hóa dữ liệu để phép tính nhỏ hơn
series = (series - series.mean()) / series.std()

In [ ]:
split_time = 80000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_val = series[split_time:]

window_size = 60
batch_size = 64
shuffle_buffer_size = 240

train_dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
val_dataset = windowed_dataset(x_val, window_size, batch_size, shuffle_buffer_size)

In [ ]:
with strategy.scope():
  model = Sequential([
      SimpleRNN(100, return_sequences=True,
                input_shape=[None, 1]),
      SimpleRNN(100),
      Dense(1)
  ])

  optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
  model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện của mô hình: {timedelta(seconds=start_time - end_time)}")

In [ ]:
# Vẽ quá trình huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Biểu đồ ở lần chạy của mình

![simple_rnn_large_dataset](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/simple_rnn_large_dataset.png?raw=true)

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
result = forecast[split_time-window_size:-1, 0]

# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([90000, len(time), -4, 4]) #Mình zoom một khoảng cho dễ coi nha.

Biểu đồ ở lần chạy của mình

![prediction_large_dataset](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_simple_rnn_large_dataset.png?raw=true)

In [ ]:
print(f"Đánh giá MAE: {mean_absolute_error(x_val, result)}")

Kết quả ở lần chạy của mình:  
> **Đánh giá MAE**: 0.2468615013962493 🌟  

Nhìn biểu đồ **loss**, mọi người thấy **loss của val** trông rất sát với **loss của train**, nhìn chung có thể nắm bắt được xu hướng một cách tương đối và có thể phát triển tiếp tục trong tương lai. Tuy nhiên, khi quan sát trên biểu đồ các giá trị dự đoán, vẫn có sự chênh lệch khá lớn, đặc biệt là ở các điểm **cực đại**. Điều này cho thấy chúng ta vẫn cần cải thiện để tối ưu thêm. 🔄  

> **Vậy nguyên nhân của nghịch lý biểu đồ loss và biểu đồ dự đoán do đâu mà ra?**  

Đó là do dữ liệu của chúng ta đã được **mã hóa về một khoảng giá trị khá nhỏ**. Vì vậy, tuy bạn thấy MAE có thể nhỏ hơn 0.1, điều này dễ dẫn đến lầm tưởng rằng mô hình vô cùng hiệu quả, nhưng thực tế thì không. Nó **tương đối lớn so với khoảng giá trị của dữ liệu**.  

### Giải pháp:  
Bạn có thể tiến hành **đảo ngược quá trình chuẩn hóa** để đưa dữ liệu về giá trị ban đầu. Lúc này, bạn có thể áp dụng công thức chuẩn hóa ngược lên MAE để tính được giá trị **MAE thực tế** và đánh giá lại độ chính xác của mô hình. 📊

# Using Other Recurrent Methods  

Tụi mình cùng khám phá, thử thêm các phương pháp, lớp hồi quy khác như **GRUs** và **LSTM** mà tụi mình đã xem qua ở chương 7 nha. 🌟  

## GRUs

In [ ]:
with strategy.scope():
  model = Sequential([
      GRU(100, return_sequences=True,
                input_shape=[None, 1]),
      GRU(100),
      Dense(1)
  ])

  optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
  model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện của mô hình: {timedelta(seconds=start_time - end_time)}")

In [ ]:
# Vẽ quá trình huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Biểu đồ ở lần chạy của mình:

![gru](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/gru.png?raw=true)

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
result = forecast[split_time-window_size:-1, 0]

# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([90000, len(time), -4, 4]) #Mình zoom một khoảng cho dễ coi nha.

Kết quả dự đoán:

![prediction_gru](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_gru.png?raw=true)

In [ ]:
print(f"Đánh giá MAE: {mean_absolute_error(x_val, result)}")

Kết quả ở lần chạy của mình:
> Đánh giá MAE: 0.3210455705409677

## LSTM

In [ ]:
with strategy.scope():
  model = Sequential([
      LSTM(100, return_sequences=True,
                input_shape=[None, 1]),
      LSTM(100),
      Dense(1)
  ])

  optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
  model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện của mô hình: {timedelta(seconds=start_time - end_time)}")

In [ ]:
# Vẽ quá trình huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Biểu đồ ở lần chạy của mình

![lstm](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/lstm.png?raw=true)

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
result = forecast[split_time-window_size:-1, 0]

# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([90000, len(time), -4, 4]) #Mình zoom một khoảng cho dễ coi nha.

Kết quả dự đoán:

![prediction_lstm](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_lstm.png?raw=true)

In [ ]:
print(f"Đánh giá MAE: {mean_absolute_error(x_val, result)}")

Kết quả ở lần chạy của mình:
> Đánh giá MAE: 0.378796568401377

Đương nhiên, mọi người cũng có thể thử thay đổi các siêu tham số khác hay thêm bớt các lớp, hoặc thử các bộ tối ưu hóa khác vào nha. 🔄  

Tại để mà nói, không có giải pháp nào phù hợp cho mọi trường hợp. Điều hoạt động tốt nhất còn phụ thuộc vào **dữ liệu** cũng như **yêu cầu** của bạn đối với dự đoán đề ra nha. 🌟  

# Using Dropout  

Đương nhiên, mọi người có thể thử thêm **dropout** vào nha, đặc biệt là với các trường hợp bị **overfitting**. Tuy nhiên, ở các lớp hồi quy - **RNN**, tụi mình sẽ có thêm một tham số mới là `recurrent_dropout` nha. 🌟  

> **Ơ, vậy nó có gì khác với dropout thông thường thế?**  

Tụi mình có thể hiểu đơn giản như này:  
- Khi sử dụng **RNN**, dữ liệu đầu vào đi qua neuron sẽ xuất dữ liệu đầu ra tại bước thời gian đó, đồng thời giá trị đó cũng được truyền tiếp cho bước thời gian tiếp theo.  
- Ý tưởng cơ bản của **dropout** là bỏ qua ngẫu nhiên một vài giá trị đầu vào hay tắt ngẫu nhiên một vài neuron đi.  
- **Recurrent_dropout** thì ngẫu nhiên bỏ qua **một vài giá trị hồi quy** được truyền qua các bước thời gian.  

Mọi người có thể coi biểu đồ dưới đây để hiểu rõ hơn nha. 📊

![rnn](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/rnn.png?raw=true)

Mọi người có thể thấy dữ liệu đầu vào của các lớp tại các bước thời gian khác nhau ha. Các giá trị hồi quy hay giá trị tái phát truyền giữa các bước được gán nhãn là **r**.  

- **Dropout**: Ngẫu nhiên bỏ qua các giá trị **x** (dữ liệu đầu vào tại các bước thời gian).  
- **Recurrent_dropout**: Ngẫu nhiên bỏ qua các giá trị **r** (giá trị hồi quy truyền qua các bước thời gian).  

Mọi người có thể tham khảo thêm tại bài báo **"*A Theoretically Grounded Application of Dropout in Recurrent Neural Networks*”** của Yarin Gal và Zoubin Ghahramani. 📖  

> **Gợi ý từ nghiên cứu của ông Gal**:  
Tuy **Dropout** mang tính ngẫu nhiên, nhưng nghiên cứu này gợi ý rằng chúng ta nên áp dụng một mẫu số lượng dropout cố định tại tất cả các bước thời gian. Điều này cũng áp dụng tương tự với **recurrent_dropout**.  

### Tiếp đến, tụi mình sẽ thử áp dụng vào code nha!  
Mình sẽ sử dụng lại mô hình có **GRU** để so sánh. 🚀

In [ ]:
with strategy.scope():
  model = Sequential([
      GRU(100, input_shape=[None, 1], return_sequences=True,
          dropout=0.1, recurrent_dropout=0.1),
      GRU(100, dropout=0.1, recurrent_dropout=0.1),
      Dense(1)
  ])

  optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
  model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện: {timedelta(seconds=end_time - start_time)}")

In [ ]:
# Vẽ quá trình huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Biểu đồ ở lần chạy của mình nha

![dropout](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/dropout.png?raw=true)

Như mọi người thấy, **chỉ số loss** giảm nhanh nhưng dần về sau tình trạng nhiễu xuất hiện ngày càng nhiều. 🌡️  

Điều này đến từ việc chúng ta sử dụng lớp **dropout**. Vì vậy, chúng ta cần phải:  
1. Điều chỉnh lượng **dropout** sao cho hợp lý.  
2. Cân nhắc điều chỉnh các tham số khác, chẳng hạn như **learning_rate**, để cải thiện sự ổn định và giảm nhiễu trong quá trình huấn luyện.  

Việc tinh chỉnh tham số này sẽ giúp mô hình đạt hiệu quả tốt hơn mà không bị ảnh hưởng bởi hiện tượng nhiễu hoặc **overfitting**. 🎯

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
result = forecast[split_time-window_size:-1, 0]

# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([90000, len(time), -4, 4]) #Mình zoom một khoảng cho dễ coi nha.

Biểu đồ ở lần chạy của mình

![prediction_dropout](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_dropout.png?raw=true)

Mọi người cũng có thể thấy trong chính chương này rằng việc dự đoán các chuỗi thời gian thường khá khó khăn và hầu như khó đạt độ chính xác hơn. 📉  

Tuy nhiên, chúng ta vẫn có thể tiến hành cải tiến các **siêu tham số** để đưa ra được giá trị **gần nhất** với thực tế. 🌟  

# Using Bidirectional RNNs

Tiếp đến tụi mình sẽ tiến hành thử nghiệm các lớp RNN theo hướng tính toán 2 chiều nha. Cái này mình train thì hết hơn 10 tiếng nên mọi người cân nhắc trước khi chạy nha.

Lưu ý ở đây mình không sử dụng `strategy` vì sau khi thử nghiệm nhiều lần, mình thấy chúng có vẻ như xung đột với bidirectional khi kết hợp kích thước 2 bên tính toán của 2 TPU lại. Do đó mình đã thử loại bỏ đi và không gặp lỗi nữa.

In [ ]:
model = Sequential([
    Bidirectional(GRU(100, input_shape=[None, 1], return_sequences=True,
                  dropout=0.1, recurrent_dropout=0.1)),
    Bidirectional(GRU(100, return_sequences=True,
                      dropout=0.1, recurrent_dropout=0.1)),
    Dense(1)
])

optimizer = SGD(learning_rate=1.5e-6, momentum=0.9)
model.compile(loss=Huber(),optimizer=optimizer, metrics=['mae'])

In [ ]:
start_time = timer.time()
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)
end_time = timer.time()

In [ ]:
print(f"Thời gian huấn luyện của mô hình: {timedelta(seconds=start_time - end_time)}")

In [ ]:
# Vẽ quá trình huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Biểu đồ ở lần chạy của mình

![bidirectional](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/bidirectional.png?raw=true)

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
result = forecast[split_time-window_size:-1, 0]

# Vẽ biểu đồ
plot_series(time_valid, x_val)
plot_series(time_valid, result)
plt.axis([90000, len(time), -4, 4]) #Mình zoom một khoảng cho dễ coi nha.

Biểu đồ kết quả dự đoán

![prediction_bidirection](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_11/prediction_bidirecitonal.png?raw=true)

In [ ]:
print(f"Đánh giá MAE: {mean_absolute_error(x_val, result)}")

Kết quả:
> Đánh giá MAE: 0.7249583964332844

Mình thấy ở kiến trúc này, có vẻ kết quả không được khả quan lắm. Chắc chúng ta cần phải tinh chỉnh nhiều hơn nữa.

# Summary  

Hi, vậy là tụi mình đã kết thúc hết chương rồi. 🎉  
Tổng kết lại những thứ mà tụi mình đã học được nha:  

- Ứng dụng **Conv1D** và **RNN** vào việc dự đoán dữ liệu dạng chuỗi.  
- Tìm hiểu về **hàm loss Huber**.  
- Hiểu và áp dụng **Dropout** và **Recurrent Dropout**. 🌟  